# **Rede Neural Convolucional**

**Importações**:

**numpy** - Biblioteca para operações numéricas.

**random** - Utilizado para gerar números aleatórios.

**matpltlib** - Biblioteca utilizada para plotar(gerar) gráficos e visualizações.

**keras** - API de redes neurais de alto nível.

**Sequential** - classe do keras utilizada para definir modelos lineares onde as camadas (layers) são adicionadas.

**Conv2D, MaxPooling2D, Flatten e Dense** - Camadas (layers) específicas necessárias para construção de redes neurais utilizando API keras.

**confusion_matrix** Biblioteca utilizada para gerar a tabela de confusão.

**seaborn** Biblioteca utilizada em conjunto ao matpltlib para plotar a matriz de confusão.

In [ ]:
import numpy as np
import random as r
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from sklearn.metrics import confusion_matrix
import seaborn as sns

Utilizando numpy, lê o arquivo de texto e o armazena em um array numpy contendo os inputs de features para treinamento e teste.

In [ ]:
X_train = np.loadtxt('/kaggle/input/imagecnn/input.csv', delimiter=',')
Y_train = np.loadtxt('/kaggle/input/imagecnn/labels.csv', delimiter=',')
X_test = np.loadtxt('/kaggle/input/imagecnn/input_test.csv', delimiter=',')
Y_test = np.loadtxt('/kaggle/input/imagecnn/labels_test.csv', delimiter=',')

Tratamento dos dados do array criado para padronizar os inputs, onde cada um é transformada em uma matriz 100x100x3.

In [ ]:
X_train = X_train.reshape(len(X_train), 100, 100, 3)
Y_train = Y_train.reshape(len(Y_train), 1)

X_test = X_test.reshape(len(X_test), 100, 100, 3)
Y_test = Y_test.reshape(len(Y_test), 1)

X_train = X_train/255.0
X_test = X_test/255.0


In [ ]:
# Inicia um modelo sequencial que permite a criação de uma pilha linear de camadas.

model = Sequential()

# Adiciona uma camada convolucional de duas dimensões e 32 filtros, 
# cada um com tamanho 3x3.

model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (100, 100, 3)))

# Adiciona uma camada de max pooling com um tamanho de pool de 2x2, 
# reduzindo as dimensões espaciais da saída da camada anterior

model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

# Reduz uma dimensão da saída da camada anterior, transformando em um array 1D.
# Essa operação é necessária para a transição de camadas convolucionais 
# para camadas totalmente conectadas.

model.add(Flatten())

# Adiciona uma camada totalmente conectada (densa) com 64 neurônios.

model.add(Dense(64, activation = 'relu'))

# Adiciona a camada de saída com um único neurônio. 
# A ativação sigmoid comprime a saída entre 0 e 1, 
# sendo adequada para problemas de classificação binária.

model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# Compila o modelo de rede neural previamente definido usando 
# o método compile() do Keras.
# O primeiro argumento especifica a função de perda utilizada para otimização.
# O segundo argumento especifica o otimizador usado para minimizar a perda.
# Especifica a métrica de avaliação a ser usada durante o treinamento e teste.

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# Treinamento do modelo de rede neural usando o método fit() do Keras.
# X_train são as features usadas para treinar o modelo.
# Y_train São os valores alvo associados às imagens de treinamento.
# epochs É o número de épocas de treinamento.
# Define o tamanho do lote de dados utilizado no treinamento.

model.fit(X_train, Y_train, epochs = 10, batch_size = 128)

In [ ]:
# Essa linha de código usa o método evaluate() do modelo Keras para
# avaliar o desempenho do modelo de rede neural nos dados de teste. 

model.evaluate(X_test, Y_test)

** Realiza uma previsão utilizando o modelo treinado em um único exemplo do conjunto de teste
**

In [ ]:
# idx = r.randint(0, len(Y_test)): Gera um número aleatório 
# entre 0 e o comprimento do conjunto de teste,

idx = r.randint(0, len(Y_test))

# plt.imshow(X_test[idx, :]): Exibe a imagem correspondente 
# ao índice aleatório selecionado. Utiliza o Matplotlib para
plt.imshow(X_test[idx, :])

# mostrar a imagem contida no conjunto de teste na posição idx.
plt.show()

# A função a seguir,faz a previsão utilizando o modelo treinado.
y_pred = model.predict(X_test[idx, :].reshape(1, 100, 100, 3))

# Transforma a saída da previsão em valores booleanos.
y_pred = y_pred > 0.5

if(y_pred == 0):
    pred = 'Cachorro'
else:
    pred = 'Gato'
    
print("Nosso modelo reconheceu este animal como um:", pred)

Para criar uma tabela de confusão, é necessário realizar previsões para todo o conjunto de teste e compará-las com os valores reais.

In [ ]:
# Realização das previsões para todo o conjunto de teste.
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  

# Transformação dos valores para binário (True/False).
Y_test_binary = (Y_test == 1)  # Considerando que 1 corresponde a 'gato'

# Criação da matriz de confusão.
conf_matrix = confusion_matrix(Y_test_binary, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',) 
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de confusão')
plt.show()